# Introduction

In this assignment, you will build a language identification classifier that distinguishes between six languages:

- [Hausa](https://en.wikipedia.org/wiki/Hausa_language)
- [Indonesian](https://en.wikipedia.org/wiki/Indonesian_language)
- [Manobo](https://en.wikipedia.org/wiki/Manobo_languages)
- [Nahuatl](https://en.wikipedia.org/wiki/Nahuatl)
- [Swahili](https://en.wikipedia.org/wiki/Swahili_language)
- [Tagalog](https://en.wikipedia.org/wiki/Tagalog_language)

Some languages can be distinguished easily, because they use different scripts. These six languages, however, use the same ([Latin](https://en.wikipedia.org/wiki/Latin_script)) script with minimal [diacritics](https://en.wikipedia.org/wiki/Diacritic) so it is difficult to hand-craft classifiers based on the presence or absence of particular characters. Indeed, unless you have linguistic training or familiarity with the languages, it is difficult to tell them apart.

How can they be distinguished? A naïve approach is to use word counts as unigram features. However, the number of possible words in a large corpus of five languages is vast. It is essential to look at something smaller — characters.

Even though the six languages use roughly the same characters, the relative frequencies of these characters vary greatly. Thus, using characters as features (unigram character models) is appealing (and fairly effective). It is also true that languages very in their *phonotactics*, the way in which consonants and vowels combine in sequence. Thus, looking at character ngrams (for small values of $n$) is also appealing (and effective). Note, however, that as the value of $n$ increases, this approach runs into the same problem as the word unigram model (sparcity). In this scenario, the model is likely to overfit.

Various kinds of classifiers can be used for this application. NB classifiers, for example, are quite effective. However, inference is slow and performance, given the same training set, is likely to be worse than other options. Simple logistic regression cannot be used because this is an n-way (multinomial) classification problem. Multinomial Logistic Regression (Softmax Regression) is a good fit.

## Summary

You will perform the following tasks:

1. Implement a training loop for Multinomial Logistic Regression.
2. Implement inference for Multinomal Logistic Regression
3. Determine the optimal order of $n$ for ngrams for MNLR trained on the training set.
4. Calculate and display a confusion matrix for a trigram model evaluated on the test set.
5. Inspect the feature weights, and display the most predictive features for each language.

# Imports
Do not change.

In [ ]:
import csv
import random

import numpy as np
import numpy.typing as npt
import numpy.testing as testing

from collections import Counter

# Helper Functions

## Metrics for Binary Classification

In [ ]:
def precision(tp: int, fp: int) -> float:
    """Computes the precision, given true positives and false positives."""
    return tp / (tp + fp)


def recall(tp: int, fn: int) -> float:
    """Computes the recall, given the true positives and false negatives."""
    return tp / (tp + fn)


def f_measure(beta: float, tp: int, fp: int, fn: int) -> float:
    """Computes the F-measure for a given beta, true positives, false positives, and false negatives."""
    return (
        (1 + beta**2)
        * (precision(tp, fp) * recall(tp, fn))
        / (beta**2 * precision(tp, fp) * recall(tp, fn))
    )


def f1(tp: int, fp: int, fn: int) -> float:
    """Computes the F1 measure for a given TP, FP, and FN."""
    return f_measure(1, tp, fp, fn)

## Micro-Averaged Metrics

In [ ]:
def micro_precision(tp: "dict[str, int]", fp: "dict[str, int]") -> float:
    """Computes micro-averaged precision."""
    tp_sum = sum(tp.values())
    fp_sum = sum(fp.values())
    return tp_sum / (tp_sum + fp_sum)


def micro_recall(tp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes micro-averaged recall."""
    tp_sum = sum(tp.values())
    fn_sum = sum(fn.values())
    return tp_sum / (tp_sum + fn_sum)


def micro_f1(tp: "dict[str, int]", fp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes micro-averaged F1."""
    mp = micro_precision(tp, fp)
    mr = micro_recall(tp, fn)
    return 2 * (mp * mr) / (mp + mr)

## Macro-Averaged Metrics

In [ ]:
def macro_precision(tp: "dict[str, int]", fp: "dict[str, int]") -> float:
    """Computes macro-averaged precision."""
    n = len(tp)
    return (1 / n) * sum([precision(tp[c], fp[c]) for c in tp.keys()])


def macro_recall(tp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes macro-averaged recall."""
    n = len(tp)
    return (1 / n) * sum([recall(tp[c], fn[c]) for c in tp.keys()])


def macro_f1(tp: "dict[str, int]", fp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes macro-averaged F1."""
    n = len(tp)
    return (
        2
        * (macro_precision(tp, fp) * macro_recall(tp, fn))
        / (macro_precision(tp, fp) + macro_recall(tp, fn))
    )

# Preprocess Data

## Loading data from files

We first need to load data from the provided TSV files. Each file is two columns, the language of the document and the document text, separated by a tab (`\t`) character. We load this data into a list of tuples, to maintain the coupling between each document and its label.

In [ ]:
def load_data(data_filename: str) -> list[tuple[str, str]]:
    with open(data_filename) as fin:
        reader = csv.reader(fin, delimiter="\t")
        language_document_tuples = [(lang, doc) for (lang, doc) in reader]
    return language_document_tuples

## Feature Extraction

We will use ngrams as features, so we need to be able to extract them.

In [ ]:
def extract_ngrams(x: str, n=3) -> "list[str]":
    """Given a string, return all character ngrams of order `n`."""
    return ["".join(s) for s in (zip(*[x[i:] for i in range(n)]))]

## Language Codes to One-Hot Vectors
And we need a function to convert a language code into a **one-hot vector** (called $\mathbf{y}$).

In [ ]:
def to_onehot_vector(lang: str, langs: list[str]) -> np.ndarray:
    y = np.zeros(len(langs))
    y[langs.index(lang)] = 1
    return y

We need to be able to convert `dict`s of ngram counts to vectors of ngram counts (using a map from ngrams to dimensions of the vectors).

In [ ]:
def vectorize_ngrams(counter: dict[str, int], feature_map: dict[str, int]) -> np.ndarray:
    """
    Given a dict of ngram counts and a map from features to indices, returns a vector of ngram counts.

    Parameters
    ----------
    counter : dict
        Counter/dict of ngram counts
    feature_map : dict
        Map from ngrams to indices

    Returns
    -------
    np.ndarray
        A vector of ngram counts
    """
    feature_vector = np.zeros(len(feature_map))
    for ngram, count in counter.items():
        if ngram in feature_map:
            feature_vector[feature_map[ngram]] = count
    return feature_vector

And putting together the conversion from text into ngrams, and vectorizing the ngrams:

In [ ]:
def vectorize_document(document: str, feature_map: dict[str, int], ngram_length: int) -> np.ndarray:
    document_ngrams = extract_ngrams(document, ngram_length)
    vector = vectorize_ngrams(Counter(document_ngrams), feature_map)
    return vector

We need separate functions for preprocessing the training observations and the dev/test observations. The former function must return a map from language names to labels as one-hot vectors as well as a map from features (ngrams) to indices of vector dimensions.

In [ ]:
def preprocess_training_observations(
    training_observations: list[tuple[str, str]], n: int = 1
) -> tuple[list[tuple[np.ndarray, np.ndarray]], dict[str, np.ndarray], dict[str, int]]:
    langs = set()
    features = set()
    obs = []

    for lang, doc in training_observations:
        langs.add(lang)
        ngrams = extract_ngrams(doc, n)
        features = features | set(ngrams)
        obs.append((lang, ngrams))
    feature_map = {feature: idx for idx, feature in enumerate(sorted(features))}
    lang_list = list(sorted(langs))
    lang_map = {lang: to_onehot_vector(lang, lang_list) for lang in lang_list}

    obs = [
        (lang_map[lang], vectorize_ngrams(Counter(ngrams), feature_map)) for (lang, ngrams) in obs
    ]

    print(f"{len(obs)} training observations.")
    return obs, lang_map, feature_map

The function for preprocessing test observations (and dev observations) takes the feature map and the language map as arguments and returns only the list of labeled observations.

In [ ]:
def preprocess_test_observations(
    test_observations, feature_map: dict[str, int], lang_map: dict[str, np.ndarray], n: int = 1
) -> tuple[list[np.ndarray], list[np.ndarray]]:
    obs = []

    for i, (lang, doc) in enumerate(test_observations):
        vectorized_doc = vectorize_document(doc, feature_map, ngram_length=n)
        try:
            obs.append((lang_map[lang], vectorized_doc))
        except KeyError:
            print(f"Unkown language {lang} at index {i}. Known languages are: {lang_map.keys()}")
    print(f"{len(obs)} test observations.")
    return obs

## Classification Function

We also need to define the softmax function.

Softmax is technically

$$\text{softmax}(z_i) = \frac{e^{z_i}}{\sum^K_{j=1} e^{z_j}}$$

However, if implemented naïvely, this is not numerically stable. Instead, we use:

$$\text{softmax}(z_i) = \frac{e^{z_i - \text{max}(z)}}{\sum^K_{j=1} e^{z_j - \text{max}(z)}}$$


In [ ]:
def softmax(z: npt.ArrayLike) -> npt.ArrayLike:
    """Compute the softmax of a vector `z`"""
    # exp(z) can get very large. For numerical stability, we subtract a vector of very large values (np.max(z)) from z.
    return np.exp(z - np.max(z)) / np.exp(z - np.max(z)).sum()

# Training the Classifier

### Compute the gradient

The formula for computing one element in our gradient is as follows (the partial derivitive of the negative log likelihood loss):

$$\frac{\partial L_{CE}}{\partial \mathbf{w}_{k,i}}=-(\mathbf{y}_k-\hat{\mathrm{y}}_k)\mathbf{x}_i$$

where $k$ is the **class** (rows of the matrix $\mathbf{w}$) and $i$ corresponds the the feature (columns of the matrix $\mathbf{w}$).

We will define a function `grad` for computing the whole gradient, a $K \times N$ matrix.

**This is the first piece of code that you'll write.**

In [ ]:
def grad(W: np.ndarray, y: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Caculates the gradient of the negative log liklihood loss, a [K * N] matrix, with respect to W.

    Parameters
    ----------
    W : np.ndarray
       A matrix of of weights
    y : np.ndarray
       The true label of the observation, expressed as a [K * N] matrix.
    x : np.ndarray
       A vector of features.

    Returns
    -------
    np.ndarray
        The gradient of the loss with respect to W.
    """
    K, N = W.shape  # K: number of classes, N: number of features
    # Compute the logits
    logits = np.dot(W, x)

    # Apply softmax to get predicted probabilities for all classes at once
    softmax_output = softmax(logits)

    # Compute the gradient for each class and feature
    grad_matrix = np.outer(softmax_output - y, x)

    return grad_matrix


## Training Loop

Iterate over the observations in the training set $n$ times (in random order). For each item, compute the one-hot vector $\mathbf{y}$ and the probability distribution $\hat{\mathbf{y}}$. Use these values to compute the gradient. Update the parameters based on the gradient. At the end of each epoch (pass through the training data), compute the true positives, false positives, and false negatives for each target class based on the current weights, and report micro-averaged precision and recall.

How you report the metrics is up to you — we will not look at what you output to STDOUT — but it is important that you do this. **Otherwise, you will not be able to determine whether your model is training.**

You can also output the loss at each step (very noisy!), each epoch, or run the classifier on the dev set and report the metrics at the end of each epoch.

Remember that the 0th column in $W$ contains the biases. You will have to insert a $1$ at the beginning of the feature vector $x$ in order to accomodate this.

In [ ]:
def train(observations: tuple[np.ndarray, np.ndarray], eta: float, epochs: int = 1) -> np.ndarray:
    """
    Given a set of observations, returns a trained multinomial LR (softmax regression) model.

    Parameters
    ----------
    observations : (np.ndarray, np.ndarray)
        Pairs consisting of a tuple of NumPy arrays (a one-hot vector encoding the ground truth language labels and a vector of features)
    eta : float
        The learning rate for parameter updates.
    epochs : int
        The number of epochs to train the model.

    Returns
    -------
    np.ndarray
        The model as a [K * N] weight matrix.
    """


    y, X = zip(*observations)
    y = np.array(y)
    X = np.array(X)
    n_samples, n_features = X.shape
    n_classes = y.shape[1]  # Assuming y is already one-hot encoded

    # Initialize the weight matrix W with small random values or zeros. Include +1 for the bias term.
    W = np.random.randn(n_classes, n_features + 1)

    for epoch in range(epochs):
        # Shuffle the dataset at the beginning of each epoch
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        X_shuffled = X[indices]
        y_shuffled = y[indices]

        total_loss = 0.0  # Accumulator for total loss in this epoch

        for i in range(n_samples):
            x = X_shuffled[i]
            true_y = y_shuffled[i]

            # Insert a 1 at the beginning of the feature vector x for the bias term
            x_bias = np.insert(x, 0, 1)

            # Compute the gradient
            gradient = grad(W, true_y, x_bias)

            # Update the weights
            W -= eta * gradient

            # Compute loss for this sample and accumulate
            loss = np.sum(-true_y * np.log(softmax(np.dot(W, x_bias))))
            total_loss += loss

        # Print average loss for this epoch
        avg_loss = total_loss / n_samples
        print(f"Epoch {epoch + 1}/{epochs}, Average Loss: {avg_loss}")


        # Here, you can add code to compute and print metrics (precision, recall) after each epoch
        # This might involve computing predictions for the training set and comparing to true_y

    return W



# Classification

The classification function is very simple.

In [ ]:
def classify(W: np.ndarray, x: np.ndarray) -> np.intp:
    """
    Return the index of the hypothesized language.

    Parameters
    ----------
    W : np.ndarray
        Weight matrix (one row for each category/language, on column for each feature)
    x : np.ndarray
        Vector of real-valuled features

    Returns
    -------
    intp
        The index of the hypothesized language.
    """
    # Insert bias term
    x = np.insert(x, 0, 1)

    # Calculate probabilities using softmax
    probabilities = softmax(np.dot(W, x))

    # Return the index with the highest probability
    return np.argmax(probabilities)

# Evaluate the Classifier

Then, a function to train and evaluate the classifier.

In [ ]:
def evaluate(train_set, test_set, eta, epochs=3):
    """
    Trains and evaluates a multinomial logistic regression model based on provided training and test datasets.
    Includes safe calculation of micro and macro F1 scores to avoid division by zero errors.
    """
    # Unpack the training and test sets
    y_train, X_train = zip(*train_set)
    y_train = np.array(y_train)
    X_train = np.array(X_train)

    y_test, X_test = zip(*test_set)
    y_test = np.array(y_test)
    X_test = np.array(X_test)

    # Train the model
    W = train(train_set, eta, epochs)

    # Initialize metrics
    tp = {i: 0 for i in range(y_test.shape[1])}
    fp = {i: 0 for i in range(y_test.shape[1])}
    fn = {i: 0 for i in range(y_test.shape[1])}

    # Predict and update TP, FP, FN
    for i in range(X_test.shape[0]):
        x = X_test[i]
        true_label = np.argmax(y_test[i])
        predicted_label = classify(W, x)

        if predicted_label == true_label:
            tp[predicted_label] += 1
        else:
            fp[predicted_label] += 1
            fn[true_label] += 1

    # Safely compute micro and macro F1 scores
    try:
        test_micro_f1 = micro_f1(tp, fp, fn)
    except ZeroDivisionError:
        test_micro_f1 = 0.0  # Default to 0 since it gives me division by zero error

    try:
        test_macro_f1 = macro_f1(tp, fp, fn)
    except ZeroDivisionError:
        test_macro_f1 = 0.0  # Default to 0 since it gives me division by zero error

    print(f"Micro F1: {test_micro_f1:.4f}")
    print(f"Macro F1: {test_macro_f1:.4f}")

    return test_macro_f1, test_micro_f1


# Putting it all together

Load the data:

In [ ]:
train_observations = load_data("train.tsv")
test_observations = load_data("test.tsv")

## Inspecting the data

Before we actually train the classifier, it's important to look at your data, and check that any assumptions you're making about it are justified. It's always useful at this point to check basic things, like:
- How many instances of train and test data do you have?
- What labels are in your data, and do those match between the train and test splits?
- What is the class balance (i.e. how many instances of each class) in your dataset? Is it balanced or unbalanced?

Output a dictionary for the train and test data that maps each label to the count of instances that have that label, e.g.:
```
{"hausa": 4000, "indonesian":...}
```

Your dictionary should be sorted in descending order of occurrence of languages.

In [ ]:
# this cell's output will be used for test 1.1
# your code here - train set
import json

# Your existing train set code to count occurrences of each label
label_counts = Counter(label for label, _ in train_observations)
label_counts_dict = dict(label_counts)

# Sort the dictionary in descending order of occurrences
sorted_label_counts = dict(sorted(label_counts_dict.items(), key=lambda item: item[1], reverse=True))

# Print the sorted dictionary in JSON format without indentation
print(json.dumps(sorted_label_counts, indent=None))


{"swahili": 377, "tagalog": 365, "manobo": 111, "hausa": 91, "nahuatl": 90, "indonesian": 86}


In [ ]:
# this cell's output will be used for test 1.2
# your code here - test set

# Your existing train set code to count occurrences of each label
label_counts = Counter(label for label, _ in test_observations)
label_counts_dict = dict(label_counts)

# Sort the dictionary in descending order of occurrences
sorted_label_counts = dict(sorted(label_counts_dict.items(), key=lambda item: item[1], reverse=True))

# Print the sorted dictionary in JSON format without indentation
print(json.dumps(sorted_label_counts, indent=None))


{"swahili": 142, "tagalog": 121, "nahuatl": 36, "manobo": 34, "indonesian": 32, "hausa": 28}


## Set hyperparameters and parameters

Before training the model, we have to set the learning rate $\eta$ and the order of the ngrams used in feature extraction.

In [ ]:
eta = 0.0005  # Do not change this.
epochs = 4  # Do not change this.
order_of_ngrams = 1  # Do change this.

## Running our training and evaluation loops

Now train your classifier, and evaluate it on the test set. Vary the number of ngrams, and observe how it changes train and test F1.

In [ ]:
train_set, lang_map, feature_map = preprocess_training_observations(
    train_observations, n=order_of_ngrams
)
test_set = preprocess_test_observations(test_observations, feature_map, lang_map, n=order_of_ngrams)


random.seed(27)
test_macro_f1, test_micro_f1 = evaluate(train_set, test_set, eta, epochs=epochs)

1120 training observations.
393 test observations.
Epoch 1/4, Average Loss: nan


<ipython-input-204-27e78a002b8f>:53: RuntimeWarning: divide by zero encountered in log
  loss = np.sum(-true_y * np.log(softmax(np.dot(W, x_bias))))
<ipython-input-204-27e78a002b8f>:53: RuntimeWarning: invalid value encountered in multiply
  loss = np.sum(-true_y * np.log(softmax(np.dot(W, x_bias))))


Epoch 2/4, Average Loss: 6.484015063598121
Epoch 3/4, Average Loss: 3.594608010156146
Epoch 4/4, Average Loss: nan
Micro F1: 0.7837
Macro F1: 0.6722


Print the tuple of the best values of `(macro_f1, micro_f1)` for the model evaluated on your test set while varying the order of the ngrams. What value of n produced the best result? Why might that be?

In [ ]:
# the output of this cell will be used for test 2.1
print((test_macro_f1, test_micro_f1))

(0.6721943405275284, 0.7837150127226463)


# Inspecting Classification Results

We've trained our classifier, and your final F1 should be pretty close to 1.0. Great job! But what does that mean for the languages you're actually classifying? Let's rewrite our evaluation code to allow us to look at our results instance-by-instance. For this, we're going to examine the results of a re-trained trigram classifier, trained for one epoch.

In [ ]:
# Re-training a trigram classifier. Do not change this.

INSPECTION_NGRAMS = 3

train_set, lang_map, feature_map = preprocess_training_observations(
    train_observations, n=INSPECTION_NGRAMS
)
test_set = preprocess_test_observations(
    test_observations, feature_map, lang_map, n=INSPECTION_NGRAMS
)

random.seed(27)
W = train(train_set, eta, epochs=1)

## evaluate it as before. Check that this looks the same!
tp, fp, fn = Counter(), Counter(), Counter()
for ref_lang_vec, x in test_set:
    ref_lang = np.argmax(ref_lang_vec)

    hyp_lang = classify(W, x)
    if hyp_lang == ref_lang:
        tp[ref_lang] += 1
    else:
        fp[hyp_lang] += 1
        fn[ref_lang] += 1
# Print metrics

print(f"macro-averaged F1:\t\t{macro_f1(tp, fp, fn):.3f}")
print(f"micro-averaged F1:\t\t{micro_f1(tp, fp, fn):.3f}")

1120 training observations.
393 test observations.
Epoch 1/1, Average Loss: 20.32535102263164
macro-averaged F1:		0.135
micro-averaged F1:		0.186


## Writing a prediction function

Write a function that takes a list of observations, and produces a list of either class indices, or class names based on a parameter. This will allow you both to look at individual results from evaluating on an existing set of data (like the test set), but also for you to evaluate your classifier on new data (i.e. any string). This will involve vectorizing the list of documents, and classifying those vectors. Once that's complete, construct an inverse language mapping, from predicted indices to the language they represent.

In [ ]:
def predict(
    documents: list[str],
    W: np.ndarray,
    feature_map: dict[str, int],
    lang_map: dict[str, np.ndarray],
    ngram_length: int,
    return_class_names=False,
):
    # Create a list (or array) of language names sorted by their order in lang_map
    # This relies on consistent ordering, so Python 3.7+ is assumed for dict insertion order preservation
    languages = list(lang_map.keys())

    predictions = []
    for doc in documents:
        vectorized_doc = vectorize_document(doc, feature_map, ngram_length)
        predicted_index = classify(W, vectorized_doc)

        if return_class_names:
            # Map the predicted index to a language name using the languages list
            predicted_lang = languages[predicted_index] if 0 <= predicted_index < len(languages) else "Unknown"
            predictions.append(predicted_lang)
        else:
            predictions.append(predicted_index)

    return predictions



Now, for each instance in the test set, print a tuple of the actual label, then the predicted label.

In [ ]:
# the output of this cell will be used for test 3.1
test_labels, test_documents = zip(*test_observations)
test_predictions = predict(
    test_documents,
    W,
    feature_map,
    lang_map,
    ngram_length=INSPECTION_NGRAMS,
    return_class_names=True,
)

print(list(zip(test_labels, test_predictions)))

[('swahili', 'manobo'), ('indonesian', 'manobo'), ('tagalog', 'swahili'), ('indonesian', 'tagalog'), ('hausa', 'tagalog'), ('nahuatl', 'indonesian'), ('hausa', 'tagalog'), ('manobo', 'swahili'), ('swahili', 'nahuatl'), ('swahili', 'nahuatl'), ('tagalog', 'manobo'), ('indonesian', 'tagalog'), ('tagalog', 'indonesian'), ('tagalog', 'indonesian'), ('swahili', 'indonesian'), ('swahili', 'swahili'), ('hausa', 'swahili'), ('indonesian', 'manobo'), ('manobo', 'manobo'), ('tagalog', 'indonesian'), ('swahili', 'tagalog'), ('swahili', 'tagalog'), ('manobo', 'swahili'), ('swahili', 'nahuatl'), ('manobo', 'manobo'), ('swahili', 'swahili'), ('tagalog', 'swahili'), ('swahili', 'tagalog'), ('swahili', 'nahuatl'), ('nahuatl', 'hausa'), ('nahuatl', 'hausa'), ('swahili', 'swahili'), ('swahili', 'tagalog'), ('swahili', 'hausa'), ('tagalog', 'swahili'), ('indonesian', 'tagalog'), ('nahuatl', 'hausa'), ('tagalog', 'tagalog'), ('tagalog', 'swahili'), ('tagalog', 'swahili'), ('tagalog', 'swahili'), ('tagalog

## Plotting a Confusion Matrix

 A _confusion matrix_ is a $k \times k$ matrix, where k is your number of classes, where the cell in position $(i,j)$ counts the number of instances that belong to class $i$ that were predicted to be in class $j$. The diagonal entries represent correct classifications; anything off of the diagonal represents an incorrect classification. The example below, from the [scikit learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html), shows a confusion matrix for a binary classification problem.

![A confusion matrix for a binary classification problem](confusion_matrix.png)


Confusion matrices can be useful to see what types of errors your classifier is making. If errors are concentrated into particular cells, it could indicate the kind of data that your classifier struggles with. Write a function to take a list of test observations, and output a numpy array that represents your classifier's confusion matrix.

In [ ]:
def get_confusion_matrix_with_labels(labels: list[str], predictions: list[str], lang_map: dict[str, np.ndarray]) -> (np.ndarray, list[str]):
    """
    Generates a confusion matrix for the given labels and predictions, along with the labels for the matrix axes.

    Parameters
    ----------
    labels : list[str]
        The actual labels of the test observations.
    predictions : list[str]
        The predicted labels by the classifier.
    lang_map : dict[str, np.ndarray]
        A dictionary mapping language names to one-hot encoded vectors.

    Returns
    -------
    tuple
        A tuple containing the confusion matrix as a numpy array and a list of language names corresponding to the labels.
    """
    # Generate a list of unique languages from lang_map keys
    languages = list(lang_map.keys())
    # Initialize a square matrix of zeros
    k = len(languages)
    confusion_matrix = np.zeros((k, k), dtype=int)

    # Create a mapping from language names to indices
    lang_to_idx = {lang: idx for idx, lang in enumerate(languages)}

    # Fill the confusion matrix
    for actual, predicted in zip(labels, predictions):
        actual_idx = lang_to_idx[actual]
        predicted_idx = lang_to_idx[predicted]
        confusion_matrix[actual_idx, predicted_idx] += 1

    return confusion_matrix, languages


Now, print your confusion matrix.

In [ ]:
# the output of this cell will be used for test 3.2
get_confusion_matrix_with_labels(test_labels, test_predictions, lang_map)


(array([[ 3,  0,  4,  6,  5, 10],
        [ 1,  2,  6,  5,  3, 15],
        [ 2,  6,  5,  2, 17,  2],
        [13,  5,  0,  1,  3, 14],
        [ 8, 17, 12, 18, 39, 48],
        [ 9, 22, 15,  2, 50, 23]]),
 ['hausa', 'indonesian', 'manobo', 'nahuatl', 'swahili', 'tagalog'])

From your confusion matrix, how many Hausa examples are misclassified as Swahili? Print each of the misclassified documents on a new line.

In [ ]:
def print_misclassified_as_swahili(test_documents: list[str], test_labels: list[str], test_predictions: list[str]):
    """
    Prints documents that were actually 'Hausa' but misclassified as 'Swahili'.

    Parameters
    ----------
    test_documents : list[str]
        The list of document texts used for testing.
    test_labels : list[str]
        The actual labels of the test documents.
    test_predictions : list[str]
        The predicted labels for the test documents.
    """
    for doc, actual, predicted in zip(test_documents, test_labels, test_predictions):
        if actual == "hausa" and predicted == "swahili":
            print(doc)

# Example usage
print_misclassified_as_swahili(test_documents, test_labels, test_predictions)



Dalilin haka kuwa sun nuna cewa ayukan da shari'a take bukata na nan a rubuce a zuciyarsu. lamirinsu kuma na yi masu shaida, tunanainsu kuma, ko dai yana kashe su, ko kuma yana karesu.
Sai aka kawo kansa bisa tire, aka mika wa yarinyar, ta kuwa kai wa mahaifiyarta.
An gicciye shi da 'yanfashi guda biyu, daya ta damansa dayan kuma ta hagunsa.
Amma da hankalinsa ya dawo, ya ce, 'Barorin mahaifina su nawa ne da suke da abinci isasshe, amma ina nan a nan, ina mutuwa sabili da yunwa!
Wannan dabban da na gani yana kama da damisa, kafafunsa kuma kamar na beyar, bakinsa kuma kamar na zaki. Wannan diragon ya ba shi karfinsa, da kursiyinsa da ikonsa mai girma na sarauta.


Can you formulate a hypothesis for why these Hausa examples are being misclassified as Swahili? Peruse the Wikipedia pages of the two languages, and inspect the data and features of the two languages. Consider reasons based in what you know about the languages, and about machine learning. Try to come up with 2-3 experiments you might run to validate your hypothesis.

## Examining Feature Weights

The classifier that we've built for softmax classification behaves in many ways like using several binary softmax classifiers stacked together. The $K \times N$ feature matrix can interpreted as a $1 \times N$ feature vector for each class. In this section, we'll examine the weights for a few of our classified languages. To get feature names out of these vectors, we'll construct an inverted feature map, that maps indices in the feature vectors back to n-grams. Then, extract the $1 \times N$ vector that corresponds to Hausa. Print the shape of the Hausa feature vector.

In [ ]:
# the output of this cell will be used for test 4.1
# Invert the feature map
inverted_feature_map = {v: k for k, v in feature_map.items()}

# Get the index corresponding to Hausa
hausa_index = list(lang_map.keys()).index("hausa")

# Use the retrieved index to access the corresponding feature vector
hausa_feature_vector = W[hausa_index]

# Print the shape of the Hausa feature vector
print("Shape of Hausa feature vector:", hausa_feature_vector.shape)


Shape of Hausa feature vector: (7942,)


Now, let's find the features that are most strongly predictive of an instance being Hausa. From the Hausa feature vector, find the names of the features with the top-10 positive values. Print your results with a tuple of (feature name, feature_weight) on each line:
```
("aaa", 0.04597442104739769)
("bbb", 0.03454984682736487)
```

In [ ]:
# the output of this cell will be used for test 4.2
# Sort the Hausa feature vector indices based on their values
top_indices = np.argsort(hausa_feature_vector)[::-1][:10]

# Print the top 10 features along with their weights

for idx in top_indices:
    feature_name = inverted_feature_map[idx]
    feature_weight = hausa_feature_vector[idx]
    print((feature_name, feature_weight))


('ì h', 4.1119423270042645)
('r M', 3.6340927749879994)
('oal', 3.6260693556138626)
('loi', 3.492575370992717)
('kuv', 3.4843143410567725)
('Mam', 3.314741520114097)
('\tKi', 3.28583461340747)
('ini', 3.2558061675166963)
('ika', 3.24587450133588)
(' Di', 3.2451698945005925)


Now, for each language, print the top-10 features. Print the name of each language, and then a list of it's top-10 features on the following line, e.g.

```
hausa
[("aaa", 0.04597442104739769)...]
indonesian
[("aaa", 0.04597442104739769)...]
```

In [ ]:
# Iterate over each language in lang_map
for lang_name in lang_map.keys():
    lang_index = list(lang_map.keys()).index(lang_name)

    # Retrieve the feature vector for the current language
    lang_vector = W[lang_index]

    # Sort the feature vector indices based on their values
    top_indices = np.argsort(lang_vector)[::-1][:10]

    # Print the name of the current language
    print(lang_name)

    # Print the top 10 features along with their weights
    top_features = []
    for idx in top_indices:
        feature_name = inverted_feature_map[idx]
        feature_weight = lang_vector[idx]
        top_features.append((feature_name, feature_weight))

    # Print the list of top features
    print(top_features)


hausa
[('ì h', 4.1119423270042645), ('r M', 3.6340927749879994), ('oal', 3.6260693556138626), ('loi', 3.492575370992717), ('kuv', 3.4843143410567725), ('Mam', 3.314741520114097), ('\tKi', 3.28583461340747), ('ini', 3.2558061675166963), ('ika', 3.24587450133588), (' Di', 3.2451698945005925)]
indonesian
[('ixt', 3.5548251508993807), ('ko?', 3.3913554320011468), ('lom', 3.3327203235724046), ('Sob', 3.208426872896643), ('k! ', 3.1842977160757657), ('ekw', 3.135631842020595), ('otl', 3.1160906223054137), ('Nig', 3.060784719629891), (" ''", 3.049404139197689), ('um ', 3.04494610483891)]
manobo
[('ria', 3.5493881873015827), ('e-b', 3.308161448631091), (' so', 3.2859171788999726), ('Ko ', 3.187335912843954), ('om;', 3.1758739166423857), (' "u', 3.170828804162903), ('t.”', 3.145984573427403), ('rk.', 3.115069358377994), (' dà', 3.103180373177725), ('te,', 3.02301659673324)]
nahuatl
[('at-', 3.4261048948110036), ('eac', 3.2705982348172613), (' Li', 3.253871248494516), ('i b', 3.2306635159016), (

Choose one of the languages, and peruse its Wikipedia page or other reliable resources to learn a bit more about the language. Do the top 10 features in that language make sense given what you've learned about the structure of the language? Why or why not? Again, consider reasons stemming both from what you've learned about the language, as well as machine learning and multinomial logistic regression specifically.

## Extra Credit

Implement at least one of the experiments you devised to test your hypothesis regarding misclassification of Hausa as Swahili. In order to get credit you must submit not just the code and results, but also clearly describe your hypothesis, the experiment, and why the experiment is suitable to test your hypothesis. Full credit will be given to hypotheses and experiments that are well thought out, explained clearly and convincingly, and backed up with suitable evidence (computed or cited).

In [ ]:
# Implement extra credit here.